In [1]:
%pip install pandas
%pip install scikit-learn
%pip install plotly
%pip install numpy
%pip install matplotlib
%pip install nbformat
%pip install opencv-python
%pip install seaborn
%pip install scikit-image
%pip install keras
%pip install tensorflow
%pip install wandb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop

import wandb
%env "WANDB_NOTEBOOK_NAME" "model_balanced.ipynb"

wandb.login()

from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint

env: "WANDB_NOTEBOOK_NAME"="model_balanced.ipynb"


In [7]:
label_df = pd.read_csv("../../../data/processed/combined/combined_label.csv", index_col=False)
data_dir = ('../../../data/processed/combined/img/')

label_df.head()

,image_name,expression_label
0,angry_0.jpg,angry
1,angry_1.jpg,angry
2,fear_2.jpg,fear
3,angry_4.jpg,angry
4,angry_5.jpg,angry


In [9]:
def load_data(dir_path, img_size):
    X = []
    y = []

    for index, row in label_df.iterrows():
        image_path = os.path.join(dir_path, row["image_name"])
        img = cv2.imread(image_path)
        img = cv2.resize(img, img_size, interpolation=cv2.INTER_AREA).astype('float32') / 255
        X.append(img)
        y.append(row["expression_label"])

    X = np.array(X)
    y = np.array(y)
    return X, y

In [10]:
IMG_SIZE= (48, 48)

X, y = load_data(data_dir, IMG_SIZE)

NameError: name 'X' is not defined

# Work from here not done

In [6]:
dummies = pd.get_dummies(fer2013_df["emotion"])
dummies.head()

,Angry,Disgust,Fear,Happy,Neutral,Sad,Surprise
0,True,False,False,False,False,False,False
1,True,False,False,False,False,False,False
2,False,False,True,False,False,False,False
3,False,False,False,False,False,True,False
4,False,False,False,False,True,False,False


In [7]:
dummies.drop(columns="Surprise", inplace=True)
dummies.head()

,Angry,Disgust,Fear,Happy,Neutral,Sad
0,True,False,False,False,False,False
1,True,False,False,False,False,False
2,False,False,True,False,False,False
3,False,False,False,False,False,True
4,False,False,False,False,True,False


In [8]:
y = dummies.values
y

array([[ True, False, False, False, False, False],
       [ True, False, False, False, False, False],
       [False, False,  True, False, False, False],
       ...,
       [ True, False, False, False, False, False],
       [False, False, False,  True, False, False],
       [False, False,  True, False, False, False]])

In [9]:
X = []
for picture in fer2013_df["pixels"].values:
    image_string = picture.split(" ")
    image_data = np.asarray(image_string, dtype=np.uint8).reshape(48, 48)
    image_rgb = np.stack([image_data] * 3, axis=-1)  # Create 3-channel image
    image_rgb_normalized = image_rgb / 255.0  # Normalize pixel values
    X.append(image_rgb_normalized)

X = np.asarray(X)

In [10]:
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=validation_ratio / (train_ratio + test_ratio)
)

print(X_train.shape)
print(X_valid.shape)
print(X_test.shape)

print(y_train.shape)
print(y_valid.shape)
print(y_test.shape)

(28709, 48, 48, 3)
(3589, 48, 48, 3)
(3589, 48, 48, 3)
(28709, 6)
(3589, 6)
(3589, 6)


In [11]:
IMG_SIZE = (48, 48)

base_model = VGG16(weights=None, include_top=False, input_shape=IMG_SIZE + (3,))

base_model.summary()

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 48, 48, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 48, 48, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 48, 48, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 24, 24, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 24, 24, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 12, 12, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 12, 12, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 12, 12, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 6, 6, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 6, 6, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 3, 3, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 3, 3, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 1, 1, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
NUM_CLASSES = 6

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(NUM_CLASSES, activation="softmax"))

In [13]:
wandb.init(
    # set the wandb project where this run will be logged
    project="Emotion Recognition",
    # track hyperparameters and run metadata with wandb.config
    config={
        "optimizer": "adam",
        "loss": "categorical_crossentropy",
        "metric": ["accuracy"],
        "epoch": 4,
        "batch_size": 32,
    },
)

In [14]:
config = wandb.config


model.compile(
    loss=config["loss"],
    optimizer=config["optimizer"],
    metrics=config["metric"],
)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_valid, y_valid),
    epochs=config["epoch"],
    batch_size=config["batch_size"],
    callbacks=[WandbMetricsLogger(log_freq=5)],
)



wandb.finish()

Epoch 1/4


wandb: ERROR Unable to log learning rate.


898/898 ━━━━━━━━━━━━━━━━━━━━ 217s 239ms/step - accuracy: 0.1959 - loss: 1440733491888128.0000 - val_accuracy: 0.1638 - val_loss: 67210886653149184.0000
Epoch 2/4
898/898 ━━━━━━━━━━━━━━━━━━━━ 215s 239ms/step - accuracy: 0.2093 - loss: nan - val_accuracy: 0.2427 - val_loss: nan
Epoch 3/4
898/898 ━━━━━━━━━━━━━━━━━━━━ 216s 241ms/step - accuracy: 0.2483 - loss: nan - val_accuracy: 0.2427 - val_loss: nan
Epoch 4/4
898/898 ━━━━━━━━━━━━━━━━━━━━ 216s 241ms/step - accuracy: 0.2528 - loss: nan - val_accuracy: 0.2427 - val_loss: nan


batch/accuracy,▁▂▂▂▂▂▂▂▂▂▃▂▂▃▃▃▄▄▅▅▆▇▇▇▇▇▇▇▇▇██▇▇▇▇▇▇▇▇
batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/loss,▁▁▁▁▁▁▁▁▁▁▃▄▅▆█
epoch/accuracy,▁▅██
epoch/epoch,▁▃▆█
epoch/loss,▁
epoch/val_accuracy,▁███
epoch/val_loss,▁
batch/accuracy,0.24857
batch/batch_step,3595
batch/loss,nan
